In [1]:
import sys
sys.path.append("../Modules/")
sys.path.append("../")
import os


import pandas as pd
import numpy as np

from configobj import ConfigObj
from configobj import flatten_errors
from validate import Validator
    
import logging

#fmt = 'logging.Formatter(''%(levelname)s_%(name)s-%(funcName)s(): - %(message)s'
fmt = '%(levelname)s_%(name)s-%(funcName)s(): - %(message)s'
logging.basicConfig(level=logging.INFO, format=fmt)
logger = logging.getLogger(__name__)


In [2]:
fileTimeList = [];
[fileTimeList.append(name) for name in os.listdir("../Data/") if name[0] is not '.'];
fileTimeList

from analysisParameters import loadParameters
from loadDataE1 import loadSessionDict
from processDataE1 import projectTrajectories

allProcData = []
for subNum, fileName in enumerate(fileTimeList):
    
    analysisParameters = loadParameters(fileName)

    # set loadProcessedData to True if you want to reprocess the file.  
    sessionDict = loadSessionDict(analysisParameters,
                                  loadParsedData = True,
                                  loadProcessedData = True)
    
    
    sessionDict['processed']['subjectNumber'] = subNum
    sessionDict['processed']['fileName'] = fileName
    
    allProcData.append(sessionDict['processed'])
    

INFO_loadDataE1-loadSessionDict(): - Loading preprocessed data for exp_data-2018-4-30-13-49 ***
INFO_loadDataE1-loadSessionDict(): - Loading preprocessed data for exp_data-2018-4-26-11-31 ***
INFO_loadDataE1-loadSessionDict(): - Loading preprocessed data for exp_data-2018-4-27-12-32 ***
INFO_loadDataE1-loadSessionDict(): - Loading preprocessed data for exp_data-2018-4-30-15-4 ***
INFO_loadDataE1-loadSessionDict(): - Loading preprocessed data for exp_data-2018-4-26-15-49 ***
INFO_loadDataE1-loadSessionDict(): - Loading preprocessed data for exp_data-2018-4-27-11-16 ***
INFO_loadDataE1-loadSessionDict(): - Loading preprocessed data for exp_data-2018-4-26-12-30 ***
INFO_loadDataE1-loadSessionDict(): - Loading preprocessed data for exp_data-2018-4-27-13-41 ***


In [3]:
allProcData = pd.concat(allProcData)
# list(sessionDict['processed'])

In [4]:
gb_sub_bl_tr = allProcData.groupby(('subjectNumber','blockNumber','trialNumber'))

In [5]:
list(allProcData)

[('IPD', ''),
 ('ballFinalPos', 'X'),
 ('ballFinalPos', 'Y'),
 ('ballFinalPos', 'Z'),
 ('ballInitialPos', 'X'),
 ('ballInitialPos', 'Y'),
 ('ballInitialPos', 'Z'),
 ('ballInitialVel', 'X'),
 ('ballInitialVel', 'Y'),
 ('ballInitialVel', 'Z'),
 ('ballMat', '0'),
 ('ballMat', '1'),
 ('ballMat', '10'),
 ('ballMat', '11'),
 ('ballMat', '12'),
 ('ballMat', '13'),
 ('ballMat', '14'),
 ('ballMat', '15'),
 ('ballMat', '2'),
 ('ballMat', '3'),
 ('ballMat', '4'),
 ('ballMat', '5'),
 ('ballMat', '6'),
 ('ballMat', '7'),
 ('ballMat', '8'),
 ('ballMat', '9'),
 ('ballPos', 'X'),
 ('ballPos', 'Y'),
 ('ballPos', 'Z'),
 ('ballRadiusM', ''),
 ('ballTTC', ''),
 ('ballVel', 'X'),
 ('ballVel', 'Y'),
 ('ballVel', 'Z'),
 ('blankDur', ''),
 ('blockNumber', ''),
 ('calibrationCounter', ''),
 ('calibrationPos', 'X'),
 ('calibrationPos', 'Y'),
 ('calibrationPos', 'Z'),
 ('cycEyeBasePoint', 'X'),
 ('cycEyeBasePoint', 'Y'),
 ('cycEyeBasePoint', 'Z'),
 ('cycEyeInHead', 'X'),
 ('cycEyeInHead', 'Y'),
 ('cycEyeInHead',

In [6]:
import matplotlib.pyplot as plt
%matplotlib notebook

In [12]:

# def plotProjectedTrajectory(tr, 
#                       halfHFOVMeters = np.tan(np.deg2rad((90/2.0))),
#                            analyzeUntilXSToArrival =  .4,
#                             stopAtXSToArrival = 0.2):

    
#     halfVFOVMeters = halfHFOVMeters / 1.77 

#     plotFr = tr.index[1:110]
#     p, ax = plt.subplots(1, 1) #sharey=True)
#     cList = ['r','g','b']
#     lineHandles = []
    
#     ballH = ax.scatter(tr[('ballOnScreen','X')][plotFr],tr[('ballOnScreen','Y')][plotFr],s=100, c='r')
#     gazeH = ax.scatter(tr[('eyeOnScreen','X')][plotFr],tr[('eyeOnScreen','Y')][plotFr],s=100, c='k',marker='+',lineWidth = 4)

#     from processDataE1 import findAnalysisWindow

#     (startFr, endFr) = findAnalysisWindow(tr,
#                                          analyzeUntilXSToArrival =  analyzeUntilXSToArrival, 
#                                           stopAtXSToArrival = stopAtXSToArrival)
#     windowFr = tr.index[startFr:endFr]
#     ax.plot(tr[('ballOnScreen','X')][windowFr],tr[('ballOnScreen','Y')][windowFr])

#     ax.text(-.85*halfHFOVMeters, .5,str('{0:.2f}').format(tr.frameTime.loc[plotFr[1]]))
#     ax.text(-.85*halfHFOVMeters, .45,str('{0:.2f}').format(tr.frameTime.loc[plotFr[-1]]))

#     ax.axis('equal')
#     ax.axes.spines['top'].set_visible(False)
#     ax.axes.spines['right'].set_visible(False)
#     ax.axes.yaxis.grid(True)
#     ax.axes.xaxis.grid(True)
#     p.set_facecolor('w')

#     plt.xlim([-halfHFOVMeters,halfHFOVMeters])
#     plt.ylim([-halfVFOVMeters,halfVFOVMeters])



In [13]:
# tr = gb_sub_bl_tr.get_group((0,1,2))
# plotProjectedTrajectory(tr)


In [ ]:
#                            analyzeUntilXSToArrival =  .4,
#                             stopAtXSToArrival = 0.2

In [9]:
def secsToEndAnalysisWindowBeforeArrival(trialData):

    ballPassesOnFr = False
    ballHitPaddleOnFr = np.where(trialData['eventFlag']=='ballOnPaddle')
    endFr = False

    initTTC = (trialData[('ballInitialPos','Z')].iloc[1] / -trialData[('ballInitialVel','Z')].iloc[1])
    
    if len(ballHitPaddleOnFr[0]) > 0:
        ballHitPaddleOnFr = ballHitPaddleOnFr[0][0]
        ballPassesOnFr = False
        endFr = ballHitPaddleOnFr
    else:
        ballHitPaddleOnFr = False
        ballPassesOnFr = np.where(trialData[('ballPos','Z')]<0)

        if len(ballPassesOnFr[0]) > 0:
            ballPassesOnFr = ballPassesOnFr[0][0]
            endFr = ballPassesOnFr
        else:
            # Sometimes the ball seems to stop in place upon collision.  I'm not sure what's going on there.
            endFr = np.where(trialData[('ballPos','Z')].diff()==0)[0][0]
    
    return trialData['frameTime'].iloc[1] + initTTC - trialData['frameTime'].iloc[endFr]


In [10]:
secsBeforeArrival = []

for (sNum,bNum,tNum), data in gb_sub_bl_tr:
    secsBeforeArrival.append(secsToEndAnalysisWindowBeforeArrival(data))

In [11]:
max(secsBeforeArrival)

0.0852876658356081

In [ ]:
.4 - max(secsBeforeArrival)

In [ ]:
tr = gb_sub_bl_tr.get_group((0,1,2))
plotProjectedTrajectory(tr,
                        analyzeUntilXSToArrival =  .4,
                        stopAtXSToArrival = max(secsBeforeArrival))
                    

# Now, relative to original TTC


In [42]:
def secsToEndAnalysisWindowBeforeArrival(trialData):

    ballPassesOnFr = False
    ballHitPaddleOnFr = np.where(trialData['eventFlag']=='ballOnPaddle')
    endFr = False

    initTTC = (trialData[('ballInitialPos','Z')].iloc[1] / -trialData[('ballInitialVel','Z')].iloc[1])

    if len(ballHitPaddleOnFr[0]) > 0:
        ballHitPaddleOnFr = ballHitPaddleOnFr[0][0]
        ballPassesOnFr = False
        endFr = ballHitPaddleOnFr

        timeOfCollision = trialData['frameTime'].iloc[endFr] - trialData['frameTime'].iloc[0]        
        endTimeRelTTC = timeOfCollision-initTTC 
    else:
        ballHitPaddleOnFr = False
        ballPassesOnFr = np.where(trialData[('ballPos','Z')]<0)

        if len(ballPassesOnFr[0]) > 0:
            ballPassesOnFr = ballPassesOnFr[0][0]
            endFr = ballPassesOnFr
        else:
            # Sometimes the ball seems to stop in place upon collision.  I'm not sure what's going on there.
            endFr = np.where(trialData[('ballPos','Z')].diff()==0)[0][0]

        timeOfCollision = trialData['frameTime'].iloc[endFr] - trialData['frameTime'].iloc[0]        
        endTimeRelTTC = timeOfCollision-initTTC 

    return endTimeRelTTC*1000


In [48]:
minSecsBeforeArrival = []

for (sNum,bNum,tNum), data in gb_sub_bl_tr:
    secsBeforeArrival.append(secsToEndAnalysisWindowBeforeArrival(data))

In [49]:
min(secsBeforeArrival)

-72.72708570610797

In [50]:
secsBeforeArrival

[-29.554993933453844,
 -2.478791285915216,
 -3.5502767987574124,
 -8.901106915031542,
 -47.49888657071155,
 -35.09809100741745,
 -43.42977209608678,
 -4.143624230336762,
 -0.6288980077089334,
 -36.51086803060699,
 -31.70060066974201,
 -46.50237696776638,
 -37.72379905539314,
 -28.69037778782868,
 -2.447877292984346,
 -40.122146425308316,
 -1.8673976146545979,
 -41.154787040805104,
 -47.04597652433185,
 -37.63878753478123,
 -36.540866044408425,
 -38.30690506677659,
 -43.78126853965103,
 -7.558236785152328,
 -39.04422086526815,
 -44.55335871244892,
 -50.45712244962552,
 -50.07728396881639,
 -46.25099295309765,
 -41.78060830531138,
 -47.734039946777386,
 -51.44369075478039,
 -7.791318942751646,
 -1.1592804481459495,
 -6.941018443658686,
 -8.742782772838353,
 -50.60773818787334,
 -35.48669713403729,
 -43.50877942814036,
 -42.95355654235378,
 -43.9753093738664,
 -41.34232144341876,
 -45.374058757796874,
 -51.035586924564,
 -45.143306600664566,
 -53.021049497755435,
 -7.238613925094395,
 -9.